## SageMaker Linear Learner - for Binary Classification


SageMaker allows us to create and train a model in a managed environment, inside containers. Therefore, you'll notice that in this stage when we move on to productionize our model pipeline, we do not use the Machine Learning framework inside our Notebook.

Inside the SageMaker managed containers, the algorithms, whether you choose the built-in algorithms, or create a container with your own algoritm, run in their own execution environment. In case of SageMaker built-in algorithms, they are implemented using MXNet as framework of choice. If you build your own algorithm, you are free to choose any framework you prefer.

In either case, you can then invoke the containers to trigger training and deployment of models using one of 3 ways:
- Using AWS SageMaker Console GUI
- Using AWS CLI
- Using Language specific API (in this case Python - Boto3)

## Configuring an S3 bucket to store training and validation data, and capture model artifacts

Before we build, train and deploy our model we need a place to store the training and validation data and capture the model artifacts.  In this case, since we are using Python, notice below that we're using the [`boto3`](https://boto3.amazonaws.com/v1/documentation/api/latest/index.html?id=docs_gateway) interface to communicate with SageMaker API, and S3 to store the data and model Artifacts.  Hence there are no other ML specific libraries imported in this notebook. 

The code below connects to an S3 bucket, creates a root folder in that bucket and a subfolder entitled data.  This is where the training and validation files will be stored.

In [1]:
import sagemaker
import boto3

#Create a SageMaker session, you'd use this to use an S3 bucket that SageMaker automatically associates for your use
session = sagemaker.Session()

#Get hold of the region you are operating, you'd use this to get ARN for algorithm container for that region
region = boto3.Session().region_name

#Get the session default bucket
s3_bucket = session.default_bucket()

#Designate location in S3 bucket where data will be staged
s3_root = 'insurance_fraud_prediction'
s3_data_prefix = s3_root + '/data'
print("Data will be staged at s3://{}/{}".format(s3_bucket, s3_data_prefix))

Data will be staged at s3://sagemaker-us-east-1-383064807500/insurance_fraud_prediction/data


## Staging, cleaning and preparing data

Since we already conducted data exploration using Scikit-Learn locally on our notebook, we now need to stage the data to be used in SageMaker managed training. 

When using SageMaker Managed training, note in the diagram below, that unlike our previous experiment, the training would not run on the Jupyter Notebook in itself. Instead, you would be supplying cleaned up data, as stage on an S3 bucket, pointing to a specific algorithm container, and SageMaker will manage procuring the training instance, running the training (in distributed mode, if chosen), de-provisioning of the traingn infrastructure, and persisiting the training output (in form of trained model file(s)) on S3 bucket.

![Image](https://miro.medium.com/max/986/1*gpgXxd6uEF22ugNWswidNw.png)


Staging the data on an S3 buclet is therefore the first step in starting a training job.

Since we didn't save the processed data frame during exploratory phase within Scikit-Learn, we'll grab the raw data and perform following steps:

- Remove unnecessary columns - during the feature engineering portion of the project, we determined that policy_number, policy_bind_date, insured_zip, incident_location, and incident_date were extraneous fields and did not help us to solve the ML problem. So these will be removed.
- Arrange columns in order, as required
- Encode columns to numeric values - Here we are taking the Yes (Y) and No (N) values for the field fraud_reported and converting these to 1 (Yes) and 0 (No). This will allow us to do math on these values later.
- Split into training and validation sets - Validation data set is not used during the actual training, and is used to test the performance of the model. In the cell below we randomly select 80% of data as training_data set and 20% as val_data set.  


Once the cell is processed you will see the results of building these two data sets with the number of features and records per set.

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split

#Load raw data from source
data = pd.read_csv("https://raw.githubusercontent.com/varun-5757/insurance_fraud/main/insurance_claims.csv").drop(columns="_c39")

#Remove unnecessary columns
colsToDelete = ["policy_number", "policy_bind_date", "insured_zip", "incident_location", "incident_date"]
for col in colsToDelete:
    del data[col]
    
#Switch target column beginning of the data frame, as is required by SageMaker Linear Learner algorithm
cols = list(data.columns)
cols = [cols[-1]]+cols[:-1]
data = data[cols]

#Change target columns to have '0' and '1' values only
data['fraud_reported'] = data['fraud_reported'].apply(lambda x: 1 if x=='Y' else 0)

#Encode all other categorical column using one-hot encoding
data = pd.get_dummies(data)

#Split transformed dataset into training and validation sets
train_data, val_data = train_test_split(data, test_size=0.2, random_state=30)    

print("Training Data : {} Features, {} Records".format(train_data.shape[1], train_data.shape[0]))
print("Validation Data : {} Features, {} Records".format(val_data.shape[1], val_data.shape[0]))

Training Data : 162 Features, 800 Records
Validation Data : 162 Features, 200 Records


## Pushing the traning and validation dataset to S3

During training, SageMaker can use data either from an [S3](https://aws.amazon.com/s3/) bucket or from [Amazon Elastic File System (EFS)](https://aws.amazon.com/efs/). 


In this case we'll use S3 for data staging. So that after pre-processing, the training and validation datasets are uploaded into S3 bucket.

The Location of data files on S3 bucket will be used as training and validation channels while setting up the predictor.

Once the code in the cell below is finished processing, there are three results:

- The training data set is saved as a csv file (train_data.csv) and placed in the data subfolder for the notebook.
- The validation data set is saved as a csv file (val_data.csv) and placed in the data subfolder for the notebook.
- A new subfoder is created for the model artififacts from the notebook called output.  This is where the models created from the notebook are stored.

In [3]:
import io

s3_resource = boto3.resource('s3')

#Upload training data file to S3
csv_buffer = io.StringIO()
train_data.to_csv(csv_buffer, header=False, index=False)
s3_train_data_key = '{}/train_data.csv'.format(s3_data_prefix)
s3_resource.Object(s3_bucket, s3_train_data_key).put(Body=csv_buffer.getvalue())
s3_train_data = 's3://{}/{}'.format(s3_bucket, s3_train_data_key)
print("Training data uploaded to: {}".format(s3_train_data))

#Upload validation data file to S3
csv_buffer = io.StringIO()
val_data.to_csv(csv_buffer, header=False, index=False)
s3_val_data_key = '{}/val_data.csv'.format(s3_data_prefix)
s3_resource.Object(s3_bucket, s3_val_data_key).put(Body=csv_buffer.getvalue())
s3_val_data = 's3://{}/{}'.format(s3_bucket, s3_val_data_key)
print("Validation data uploaded to: {}".format(s3_val_data))

output_location = 's3://{}/{}/output'.format(s3_bucket, s3_root)
print('Model artifacts will be uploaded to: {}'.format(output_location))


Training data uploaded to: s3://sagemaker-us-east-1-383064807500/insurance_fraud_prediction/data/train_data.csv
Validation data uploaded to: s3://sagemaker-us-east-1-383064807500/insurance_fraud_prediction/data/val_data.csv
Model artifacts will be uploaded to: s3://sagemaker-us-east-1-383064807500/insurance_fraud_prediction/output


## Model Selection

Since our initial experiment showed promising results on this dataset using logistic regression within Scikit-Learn, we can implement the same using SageMaker using Linear Learner model with Binary classifier predictor type. 

SageMaker's linear-learner algorithm performs logistic regression assuming a linear separation boundary between two classes.

![Image](https://miro.medium.com/max/1680/0*gKOV65tvGfY8SMem.png)

## Obtaining the Docker container image for the linear-learner algorithm and generating unique names for model artifacts

First step therefore is to get hold of the container image for `linear-learner` in the region we are operating.  

As a first step, we now need to obtain the container image for the `linear-learner` algorithm in the region we are operating in.  

Each training job also needs a unique name, which we create by appending current timestamp with a name of our choice.  

In [12]:
import time
from datetime import datetime
from sagemaker import get_execution_role
from sagemaker.amazon.amazon_estimator import get_image_uri

# Obtain container image for Linear Learner Algorithm
#container = get_image_uri(region, 'linear-learner')

image_uris.retrieve(region=boto3.Session().region_name, framework='linear-learner', version='1')
print("Algorithm container: {}".format(container))

# Obtain SageMaker execution role that will be used to invoke Sagemaker commands
role = get_execution_role()

# SageMaker client using Python boto3 API
sagemaker = boto3.client('sagemaker')

#Generate unique names for artefacts, with timestamp, for easy tracking
pipeline_name = 'insurance-fraud-predictor'
runtime = datetime.now().strftime("%Y-%m-%d-%H-%M-%S-%f")[:-3]
run_name = "{}-{}".format(pipeline_name, runtime)

Algorithm container: 382416733822.dkr.ecr.us-east-1.amazonaws.com/linear-learner:1


## Model training

With the image URI readily available, not a single line of Model definition or training code needs to be written.
Instead we could follow one of two simple approaches:
    
1. Create an instance of SageMaker Estimator, which provides a familar programming interface common to most other Machine Language frameworks, such as Scikit-Learn, and then invoke a `fit` operation. This is scikit-learn emulation.

2. Use `boto3` API to create a sagemaer training job, supplying the training image, hyperparameters, input and output data and resource configurations.

Both approaches produce same result, which is to train a model on a specified dataset using the specified algorithm container, and save the model artifact to an S3 bucket.

In this case, we'll use the later (Example using the former approach can be found [here](https://github.com/awslabs/amazon-sagemaker-examples/blob/master/introduction_to_amazon_algorithms/linear_learner_mnist/linear_learner_mnist.ipynb))  See the code cell 7 and 8 in this notbook for details on how to use the SageMaker Estimator, including the fit operation.

**The cell below addresses the following objectives from the Algorithms Module:**

- **Configure and initiate a training job**
- **Associate the training job with the appropriate S3 bucket**
- **Select the desired or default parameters**

In [13]:
response = sagemaker.create_training_job(
    TrainingJobName=run_name,
    HyperParameters={
        'feature_dim': str(data.shape[1]-1),
        'predictor_type': 'binary_classifier',
        'binary_classifier_model_selection_criteria': 'f_beta',
        'optimizer': 'sgd',
        'loss': 'logistic',
        'normalize_data': 'auto',
        'epochs': '20',
        'mini_batch_size': '200'
    },
    AlgorithmSpecification={ 'TrainingImage': container, 'TrainingInputMode': 'File'},    
    RoleArn=role,
    InputDataConfig=[
        {
            'ChannelName': 'train',
            'ContentType': 'text/csv',
            'DataSource': {
                'S3DataSource': {'S3DataType': 'S3Prefix','S3Uri': s3_train_data,'S3DataDistributionType': 'FullyReplicated'}
            },
            'CompressionType': 'None',
            'RecordWrapperType': 'None'
        },
        {
            'ChannelName': 'validation',
            'ContentType': 'text/csv',
            'DataSource': {
                'S3DataSource': {
                    'S3DataType': 'S3Prefix',
                    'S3Uri': s3_val_data,
                    'S3DataDistributionType': 'FullyReplicated'
                }
            },
            'CompressionType': 'None',
            'RecordWrapperType': 'None'
        },        
    ],
    OutputDataConfig={'S3OutputPath': output_location},
    ResourceConfig={
        'InstanceType': 'ml.m4.xlarge',
        'InstanceCount': 1,
        'VolumeSizeInGB': 10
    },
    StoppingCondition={'MaxRuntimeInSeconds': 86400},
    Tags=[{
            'Key': 'Name',
            'Value': '{}-training'.format(run_name)
        }]    
)

print("Training Job - {} started".format(response['TrainingJobArn']))

Training Job - arn:aws:sagemaker:us-east-1:383064807500:training-job/insurance-fraud-predictor-2021-03-02-15-04-37-940 started


In this particular example we are not using any Hyperparameter optimization, instead we are using some hyperparameters that work best on this dataset, based on our previous experiment.

Where manually deciding the right set of hyper-parameters is difficult, and the default set of hyparameters do not produce required result, we can use [Automated Model Tuning](https://docs.aws.amazon.com/sagemaker/latest/dg/automatic-model-tuning-how-it-works.html) feature of SageMaker to arrive at an optimal set fo Hyperparameters.  


Depending upon the complexity of the model and size of training datset, it will take few minutes to few hours for the model to be ready. 

We therefore wait until the model get trained. In this case, since we are using a very small datset, the training should not be more than 2-3 minutes.

In [15]:
status='InProgress'
step = 0
sleep = 10
print("Training job {} - {} - Time Elapsed: {} seconds".format(run_name, status,step*sleep))
while status != 'Completed' and status != 'Failed':
    response = sagemaker.describe_training_job(
        TrainingJobName=run_name
    )
    status = response['TrainingJobStatus']
    time.sleep(sleep)
    step = step+1
    print("Training job {} - {} - Time Elapsed: {} seconds".format(run_name, status,step*sleep))

Training job insurance-fraud-predictor-2021-03-02-15-04-37-940 - InProgress - Time Elapsed: 0 seconds
Training job insurance-fraud-predictor-2021-03-02-15-04-37-940 - Completed - Time Elapsed: 10 seconds


## Model Hosting and Inference

Now that our model is trained, we can now deploy it behind an HTTP endpoint.

Take a look a the graphic below.  The graphic illustrates model deployment with SageMaker Hosting Services with an HTTPs endpoint where your machine learning model is able to provide inferences.

![Image](https://docs.aws.amazon.com/sagemaker/latest/dg/images/sagemaker-architecture.png)

There are three steps involved in having a trained model deployed:

1. Creating a `Model` definition that acts as a placeholder, specifying location of a model artifact (on Amazon S3 or Amazon Elastic File System) and ECS registry path of inference code image.


2. Creating an `Endpoint Configuration`, using the `Model` definition above that defines configurations such as number and types of instances to be used, fraction of traffic to be handled by the endpoint (once deployed), and specifies whether or not [Elastic inference](https://docs.aws.amazon.com/sagemaker/latest/dg/ei.html) (a feature that allows you to speed up throughput and decrease latency of getting real-time inferences) is to be used.


3. Creating an `Endpoint`, using the `Endpoint Configuration` defined above, which creates a SageMaker managed container with the inference code served by a Flask web application.

Similar to the model training step, in this example, we'll use `Boto3` API to execute the 3 steps listed above.

The first step creates the `Model` definition. Once the code in the cell below is successfully processed, the model definition ARN (Amazon Resource Name) will be created and displayed below.

In [16]:
if status == 'Completed':
    runtime = datetime.now().strftime("%Y-%m-%d-%H-%M-%S-%f")[:-3]
    model_name = "{}-{}".format(pipeline_name, runtime)    
    response = sagemaker.create_model(
        ModelName=model_name,
        PrimaryContainer={
            'Image': container,
            'ModelDataUrl': "{}/{}/output/model.tar.gz".format(output_location, run_name),
            'Environment': {
                'string': 'string'
            }
        },
        ExecutionRoleArn=role,
        Tags=[
            {
                'Key': 'Name',
                'Value': model_name
            }
        ]
    )
    
print("Model definition - {} created".format(response['ModelArn']))

Model definition - arn:aws:sagemaker:us-east-1:383064807500:model/insurance-fraud-predictor-2021-03-02-15-24-55-045 created


The Second step creates the `Endpoint Configuration`.  Once the code in the cell below is successfully processed, the endpoint configuration ARN (Amazon Resource Name) will be created and displayed below.

In [17]:
runtime = datetime.now().strftime("%Y-%m-%d-%H-%M-%S-%f")[:-3]
endpoint_config_name = "{}-{}".format(pipeline_name, runtime) 

response = sagemaker.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=[
        {
            'VariantName': 'default',
            'ModelName': model_name,
            'InitialInstanceCount': 1,
            'InstanceType': "ml.t2.medium",
            'InitialVariantWeight': 1
        },
    ],
    Tags=[
        {
            'Key': 'Name',
            'Value': endpoint_config_name
        }
    ]
)

print("Endpoint configuration - {} specified".format(response['EndpointConfigArn']))

Endpoint configuration - arn:aws:sagemaker:us-east-1:383064807500:endpoint-config/insurance-fraud-predictor-2021-03-02-15-25-35-101 specified


Finally in the third step of deployment we deploy the `Model` behind an `Endpoint`, using the `Endpoint Configuration`.

Behind the scenes SageMaker provisions instance(s) of desired type, pulls the model artifact, and configures the runtime, which takes 11 to 12 minutes.  You will see the status of the endpoint creation process updated every 30 seconds beneath the cell.  Once the endpoint creation process is complete the status will change to InService.

In [18]:
runtime = datetime.now().strftime("%Y-%m-%d-%H-%M-%S-%f")[:-3]
endpoint_name = "{}-{}".format(pipeline_name, runtime) 

response = sagemaker.create_endpoint(
    EndpointName=endpoint_name,
    EndpointConfigName=endpoint_config_name,
    Tags=[
        {
            'Key': 'string',
            'Value': endpoint_name
        }
    ]
)
print("Endpoint - {} deployment started".format(response['EndpointArn']))

status='Creating'
step = 0
sleep = 30
print("{} Enpoint {} - Time Elapsed: {} seconds".format(status,endpoint_name,step*sleep))
while status != 'InService' and status != 'Failed' and status != 'OutOfService':
    response = sagemaker.describe_endpoint(
        EndpointName=endpoint_name
    )
    status = response['EndpointStatus']
    time.sleep(sleep)
    step = step+1
    print("{} Enpoint {} - Time Elapsed: {} seconds".format(status,endpoint_name,step*sleep))



Endpoint - arn:aws:sagemaker:us-east-1:383064807500:endpoint/insurance-fraud-predictor-2021-03-02-15-32-13-384 deployment started
Creating Enpoint insurance-fraud-predictor-2021-03-02-15-32-13-384 - Time Elapsed: 0 seconds
Creating Enpoint insurance-fraud-predictor-2021-03-02-15-32-13-384 - Time Elapsed: 30 seconds
Creating Enpoint insurance-fraud-predictor-2021-03-02-15-32-13-384 - Time Elapsed: 60 seconds
Creating Enpoint insurance-fraud-predictor-2021-03-02-15-32-13-384 - Time Elapsed: 90 seconds
Creating Enpoint insurance-fraud-predictor-2021-03-02-15-32-13-384 - Time Elapsed: 120 seconds
Creating Enpoint insurance-fraud-predictor-2021-03-02-15-32-13-384 - Time Elapsed: 150 seconds
Creating Enpoint insurance-fraud-predictor-2021-03-02-15-32-13-384 - Time Elapsed: 180 seconds
Creating Enpoint insurance-fraud-predictor-2021-03-02-15-32-13-384 - Time Elapsed: 210 seconds
Creating Enpoint insurance-fraud-predictor-2021-03-02-15-32-13-384 - Time Elapsed: 240 seconds
Creating Enpoint ins

## Model Testing

Having our linear logistic regression model deployed essentially completes the deployment cycle, and we are now ready to test the accuracy of the model.


To do so, we'll use the validation data that was saved separately in S3.  Once the code in the cell is processed, the validation data set will be loaded into this notebook as indicated by the message "200 Test record loaded."

In [19]:
import s3fs
import json
test_data = pd.read_csv(s3_val_data, header=None)
print("{} Test records loaded.".format(test_data.shape[0]))

200 Test records loaded.


With each record in the validation data set, we conduct the following steps:

1. Create a data dump without the target column


2. Transform it into an input record, following the format expected by the corresponding inference code.


3. Invoke the deployed endpoint using SageMaker Runtime client.


4. Compare the predicted label returned with the expected label, as specified in the target column.


5. Use the comparison result to appropriately classify the out come as one of: 
    - True Negative
    - False Positive
    - True Positive
    - False Negative
    
Once the processing of the cell below is complete you will see a list of each record with the target and predicted values.  Where 1 indicates fraud and 0 indicates no fraud. 

In [20]:
sagemaker_runtime = boto3.client('sagemaker-runtime')

true_negative = 0
false_negative = 0
false_positive = 0
true_positive = 0

#Repeat for each validation record
for i in range(test_data.shape[0]):
    
    #Separate the taregt label (expected outcome)
    sample_target = int(test_data.loc[i, 0])
    
    #Transform rest of the record into input feature set
    sample_features = test_data.loc[i, 1:].to_frame().transpose().to_csv(index=False, header=False).split(',')
    sample_request = json.dumps({"instances": [{"features": sample_features}]})
    
    #Invoke inference endpoint
    response = sagemaker_runtime.invoke_endpoint(
        EndpointName=endpoint_name,
        Body=sample_request
    )
    
    #Compare target vs. predicted
    predictions = json.loads(response['Body'].read().decode())['predictions']
    sample_predicted = int(predictions[0]['predicted_label'])
    print("Test-{}: Target - {}, Predicted - {}".format(i+1, sample_target, sample_predicted))
    
    if sample_target == 0:
        if sample_predicted == 0:
            true_negative = true_negative + 1
        else:
            false_positive = false_positive + 1
    else:
        if sample_predicted == 1:
            true_positive = true_positive + 1
        else:
            false_negative = false_negative + 1

Test-1: Target - 0, Predicted - 0
Test-2: Target - 0, Predicted - 0
Test-3: Target - 0, Predicted - 0
Test-4: Target - 0, Predicted - 0
Test-5: Target - 0, Predicted - 1
Test-6: Target - 0, Predicted - 0
Test-7: Target - 0, Predicted - 1
Test-8: Target - 0, Predicted - 0
Test-9: Target - 0, Predicted - 0
Test-10: Target - 0, Predicted - 0
Test-11: Target - 1, Predicted - 0
Test-12: Target - 0, Predicted - 0
Test-13: Target - 0, Predicted - 0
Test-14: Target - 0, Predicted - 1
Test-15: Target - 0, Predicted - 0
Test-16: Target - 1, Predicted - 1
Test-17: Target - 1, Predicted - 1
Test-18: Target - 0, Predicted - 0
Test-19: Target - 0, Predicted - 0
Test-20: Target - 1, Predicted - 1
Test-21: Target - 0, Predicted - 1
Test-22: Target - 1, Predicted - 1
Test-23: Target - 1, Predicted - 0
Test-24: Target - 0, Predicted - 1
Test-25: Target - 0, Predicted - 0
Test-26: Target - 0, Predicted - 0
Test-27: Target - 0, Predicted - 0
Test-28: Target - 1, Predicted - 0
Test-29: Target - 0, Predicte

In [21]:
print("Test Results:")
print("True Negative = {}".format(true_negative))
print("False Positive = {}".format(false_positive))
print("True Positive = {}".format(true_positive))
print("False Negative = {}".format(false_negative))

Test Results:
True Negative = 122
False Positive = 32
True Positive = 37
False Negative = 9


With the counts of prediction outcome computed, we can use the simple formulas to determine the Precision, Recall and F1 score for the model.

![Image](https://miro.medium.com/max/1520/1*OhEnS-T54Cz0YSTl_c3Dwg.jpeg)

![Image](https://miro.medium.com/max/888/1*7J08ekAwupLBegeUI8muHA.png)



![Image](https://miro.medium.com/max/564/1*T6kVUKxG_Z4V5Fm1UXhEIw.png)

In [22]:
precision = true_positive/(true_positive+false_positive)
recall = true_positive/(true_positive+false_negative)
f1 = 2 * precision * recall / (precision + recall)

print("Precision : %.2f" % precision)
print("Recall : %.2f" % recall)
print("F1 Score : %.2f" % f1)

Precision : 0.54
Recall : 0.80
F1 Score : 0.64


## Comparison of Results

Notice that, even without any Hyperparameter tuning, using the SageMaker Linear Learner, our first attempt yielded results comparable to that obtained by using Scikit Learn Logistic Regression classifier.

|        	| Scikit-Learn Logistic Regression | SageMaker Linear Learner |
|----------	|:-------------:	|------:	|
| Precision |  0.63 	| 0.54	|
| Recall 	|    0.55   	|   0.80	|
| F1 Score 	| 0.59	|    0.64 	|

## Cleanup

When a model is deployed behind endpoint using SageMaker's managed deployment environments, unlike local test environments, you'll end up having a persistent artifact, backed by an instance that will be running until terminated.

Keep in mind that you by the usage for running instances. Therefore for any exercises, such as this one, that you conduct as proof of concept, it is very important to destroy the artifacts when not needed.

`Boto3` API provides convenient methods to remove all such artefacts, thereby saving you on cost.

In [23]:
sagemaker.delete_endpoint(EndpointName = endpoint_name)
sagemaker.delete_endpoint_config(EndpointConfigName = endpoint_config_name)
sagemaker.delete_model(ModelName = model_name)

{'ResponseMetadata': {'RequestId': 'f7c75e7a-d2cf-44d8-9fba-4e7aa7a072cb',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'f7c75e7a-d2cf-44d8-9fba-4e7aa7a072cb',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '0',
   'date': 'Tue, 02 Mar 2021 15:53:52 GMT'},
  'RetryAttempts': 0}}